In [ ]:
#!pip install openai
#!pip install pydub
#!pip install soundcard
#!pip install soundfile

In [ ]:
TOKEN = 'XXXXX'

In [ ]:
# Recording duration
RECORD_MIN =  int(input("Enter time duration in minutes: "))*60 # [min]. duration recording audio.
OUTPUT_FILE_NAME = "output.wav"    # file name.
SAMPLE_RATE = 48000              # [Hz]. sampling rate

In [ ]:
import soundcard as sc
import soundfile as sf
from pydub import AudioSegment
import datetime
import os
import openai
openai.api_key = TOKEN

In [ ]:
with sc.get_microphone(id=str(sc.default_speaker().name), include_loopback=True).recorder(samplerate=SAMPLE_RATE) as mic:
    # record audio with loopback from default speaker.
    print("Recording.....n")
    data = mic.record(numframes=SAMPLE_RATE*RECORD_MIN)
    
    # change "data=data[:, 0]" to "data=data", if you would like to write audio as multiple-channels.
    sf.write(file=OUTPUT_FILE_NAME, data=data[:, 0], samplerate=SAMPLE_RATE)
    print("End Recording")

In [ ]:
# Initialize an empty string to hold the transcript
transcript = ""

# load audio file
sound = AudioSegment.from_file(OUTPUT_FILE_NAME, format='wav')

# Set the segment length（unit：ms）
segment_length = 100000

# segment the audio file to multiple file
for i, chunk in enumerate(sound[::segment_length]):
    # Set the file name
    chunk.export(f'output_{i}.wav', format='wav')
    audio_file = open(f'output_{i}.wav', "rb")
    transcription = openai.Audio.transcribe("whisper-1", audio_file)
    # Concatenate the resulting text with the existing transcript
    transcript += transcription.to_dict().get('text') + ' '
    
# Print the final transcript
#print(transcript)

In [ ]:
transcript_ary = []

ret = ''
for script in transcript.split():
    ret = ret + ' ' + script
    if len(ret) > 1000:
        transcript_ary.append(ret)
        ret = ''
transcript_ary.append(ret)
#print(transcript_ary)

In [ ]:
# Get the current date and time
now = datetime.datetime.now()
# Format the date and time as a string
timestamp = now.strftime("%Y-%m-%d_%H-%M-%S")
# Set the file name using the timestamp
filename = f"Raw_transcript_response_{timestamp}.txt"

# Print the name of the saved file
print(f"Data saved to {filename}.")

# Open the file in write mode and save the data
with open(filename, "w", encoding="utf-8") as f:
    for item in transcript_ary:
        f.write(item + "\n")

In [ ]:
result_ary = []

for t in transcript_ary:
    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": "Please be a professional writer for article summarization and refine, rewrite, and summarize the following text in english"},
        {"role": "user", "content": t}
      ]
    )

    result_ary.append(completion.choices[0].message)

In [ ]:
# Initialize an empty string to hold the transcript
final_result = ""
for res in result_ary:
    # Concatenate the resulting text with the existing transcript
    final_result += res.get('content') + ' '
    print(res.get('content'))

In [ ]:
# Get the current date and time
now = datetime.datetime.now()
# Format the date and time as a string
timestamp = now.strftime("%Y-%m-%d_%H-%M-%S")
# Set the file name using the timestamp
filename = f"Summary_response_{timestamp}.txt"

# Open the file in write mode and save the data
with open(filename, "w") as f:
    f.write(final_result)

# Print the name of the saved file
print(f"Data saved to {filename}.")